## 14: Accelerating Geospatial Searches with FAISS

**Goal:** To demonstrate the utility of Facebook AI's Similarity Search (`faiss`) library for dramatically accelerating nearest-neighbor searches in a geospatial context. For large-scale problems, this is a critical optimization.

**Methodology:**
1.  **Create a Large-Scale Scenario:** Generate a large number of demand points (patients) and a hierarchical set of healthcare facilities.
2.  **Benchmark Traditional Search:** Time a standard nearest-neighbor search using `scipy` or `sklearn` as a baseline.
3.  **Implement FAISS:** Build a `faiss` index for the facility locations.
4.  **Benchmark FAISS Search:** Run the same search using the `faiss` index and compare the execution time.
5.  **Hierarchical FAISS Search:** Show how to build separate indexes for each facility type (GP, CDC, Acute) to enable efficient, targeted queries based on the required level of care.

### 1. Setup and Library Imports

**Important:** This notebook requires `faiss-cpu`. Install it by running this command in your terminal (with your `spatial-ml-env` activated), and then **restart the Jupyter kernel**.

```bash
pip install faiss-cpu
```

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import faiss
import time
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.neighbors import BallTree

### 2. Create a Large-Scale Synthetic Dataset

To showcase the speed of `faiss`, we need a non-trivial amount of data. We'll create 100,000 demand points.

In [ ]:
np.random.seed(42)

# 1. Facilities (GP, CDC, Acute Hospital)
facilities_data = []
types = {'GP Practice': 20, 'CDC/Community': 5, 'Acute Hospital': 2}
for f_type, count in types.items():
    for i in range(count):
        facilities_data.append({
            'name': f'{f_type} {i+1}',
            'type': f_type,
            'geometry': Point(np.random.uniform(0, 100), np.random.uniform(0, 100))
        })
facilities_gdf = gpd.GeoDataFrame(facilities_data, crs="EPSG:32630")

# 2. Demand Points (Patients)
num_demand_points = 100000
demand_points_gdf = gpd.GeoDataFrame(
    geometry=[Point(x, y) for x, y in zip(np.random.uniform(0, 100, num_demand_points), 
                                             np.random.uniform(0, 100, num_demand_points))],
    crs="EPSG:32630"
)

print(f"Created {len(facilities_gdf)} facilities and {len(demand_points_gdf)} demand points.")

### 3. Benchmark: Traditional Nearest-Neighbor Search

We use `sklearn.neighbors.BallTree`, a standard and efficient library for this task, as our baseline.

In [ ]:
start_time = time.time()

# Prepare coordinate arrays
facility_coords = np.array([pt.coords[0] for pt in facilities_gdf.geometry])
demand_coords = np.array([pt.coords[0] for pt in demand_points_gdf.geometry])

# Build the tree
tree = BallTree(facility_coords)

# Query the tree for the single nearest neighbor (k=1)
distances, indices = tree.query(demand_coords, k=1)

end_time = time.time()
traditional_time = end_time - start_time

print(f"Traditional BallTree search took: {traditional_time:.4f} seconds")

### 4. Benchmark: FAISS Accelerated Search

`faiss` requires a few steps: (1) Prepare data as a `float32` numpy array, (2) Build an index, (3) Add the facility vectors to the index, (4) Search.

In [ ]:
start_time = time.time()

# Prepare coordinate arrays in the required float32 format
facility_vectors = np.array([pt.coords[0] for pt in facilities_gdf.geometry]).astype('float32')
demand_vectors = np.array([pt.coords[0] for pt in demand_points_gdf.geometry]).astype('float32')

d = facility_vectors.shape[1]  # Dimension of the vectors (should be 2 for X,Y)

# Build the index. IndexFlatL2 is a brute-force L2 distance search.
index = faiss.IndexFlatL2(d)
print(f"FAISS index is trained: {index.is_trained}")

# Add the facility vectors to the index
index.add(facility_vectors)
print(f"Total vectors in index: {index.ntotal}")

# Search for the 1 nearest neighbor (k=1)
D, I = index.search(demand_vectors, k=1)

end_time = time.time()
faiss_time = end_time - start_time

print(f"\nFAISS search took: {faiss_time:.4f} seconds")
print(f"\nSpeed-up: {traditional_time / faiss_time:.2f}x faster")

### 5. Hierarchical Search with FAISS

A major advantage is the ability to create targeted indexes. If a patient needs an Acute Hospital, we should only search against that specific subset of facilities.

In [ ]:
# Create separate GeoDataFrames for each facility type
gp_gdf = facilities_gdf[facilities_gdf['type'] == 'GP Practice']
cdc_gdf = facilities_gdf[facilities_gdf['type'] == 'CDC/Community']
acute_gdf = facilities_gdf[facilities_gdf['type'] == 'Acute Hospital']

# --- Build an index for each type ---
def build_index(gdf):
    vectors = np.array([p.coords[0] for p in gdf.geometry]).astype('float32')
    d = vectors.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(vectors)
    return index

index_gp = build_index(gp_gdf)
index_cdc = build_index(cdc_gdf)
index_acute = build_index(acute_gdf)

print(f"GP Index has {index_gp.ntotal} vectors.")
print(f"CDC Index has {index_cdc.ntotal} vectors.")
print(f"Acute Hospital Index has {index_acute.ntotal} vectors.")

# --- Scenario: Find the nearest Acute Hospital for the first 5 patients ---
query_vectors = np.array([pt.coords[0] for pt in demand_points_gdf.head(5).geometry]).astype('float32')

distances_to_acute, indices_in_acute_gdf = index_acute.search(query_vectors, k=1)

# Get the actual name of the found hospital
found_hospital_names = acute_gdf.iloc[indices_in_acute_gdf.flatten()]['name'].values

for i in range(5):
    print(f"Patient {i} is closest to: {found_hospital_names[i]}")

### 6. Conclusion and Use Cases

As demonstrated, `faiss` offers a massive performance improvement for nearest-neighbor searches, a foundational operation in spatial analysis.

* **Key Advantage:** Speed at scale. While the setup is slightly more complex than a one-line `BallTree` query, the speed-up is essential when dealing with hundreds of thousands or millions of points.
* **Hierarchical Planning:** Building targeted indexes for different facility types is a clean and highly efficient way to model tiered service systems.
* **Practical Applications:**
    - **Large-Scale Allocation Models:** In optimization problems that require calculating a full distance matrix (demand points x facilities), `faiss` can pre-calculate the `k` nearest candidates for each demand point, dramatically reducing the problem size.
    - **Interactive Dashboards:** For any web-based tool or dashboard where a user can click a point and expect an instant result for the nearest facility, `faiss` is the ideal backend engine.
    - **Data Enrichment:** When joining large datasets based on proximity, `faiss` can quickly find the nearest feature from one dataset for every feature in another.